In [ ]:
!pip install ipympl

In [4]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 

%matplotlib widget

#inspired by https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

In [6]:
data = pd.read_excel('data.xlsx') 

In [ ]:
data.info

In [11]:
X = data.iloc[:, 0:162].values #features 
y = data.iloc[:, 162].values #labels

np.save('X', X) 
np.save('y', y)

### Load Data + Analysis

In [12]:
X = np.load('X.npy')
y = np.load('y.npy')

In [14]:
fig = plt.figure()
_ = plt.hist(y)
plt.xlabel('Decision')
plt.ylabel('# Occurence')
plt.suptitle('Histogram of Decision')
plt.show()

## On vérifie que la distribution est "logique" 
# et ça se vérifie car plus grande distribution en 0 (donc ne rien faire) ok mais attention 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
#on peut normaliser les données ici
xMin = -1.2
xMax = 9.6
yMin = -5.4
yMax = 5.4

X[:,0] = (X[:,0]-xMin)/(xMax-xMin) #position en x [0,1]
X[:,1] = (X[:,1]-yMin)/(yMax-yMin) #position en y [0,1]
X[:,2] = X[:,2]/360 #degre -> [0 1]

In [17]:
X.shape

(4729, 162)

### Random Forest

In [18]:
# import
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [19]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
cf = RandomForestClassifier(n_jobs=-1)

In [20]:
# Number of trees in random forest
n_estimators = [10, 100, 1000]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [10, 100]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
params_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [21]:
grid_search = GridSearchCV(estimator=cf, param_grid=params_grid, cv=5)
grid_search.fit(X, y)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=-1,
                                              oob_score=False,
                                              random_

## Save Model

In [24]:
import pickle

# Save to file in the current working directory
pkl_filename = "pickle_model.pkl"

with open(pkl_filename, 'wb') as file:
    pickle.dump(grid_search, file)
